In [105]:
import pandas as pd

In [106]:
crop_df = pd.read_csv("crop.csv")
rain_df = pd.read_csv("rain.csv")

In [107]:
crop_df.shape

(345407, 10)

In [108]:
rain_df.shape

(4188, 19)

In [109]:
list(crop_df.columns)

['State',
 'District',
 'Crop',
 'Year',
 'Season',
 'Area',
 'Area Units',
 'Production',
 'Production Units',
 'Yield']

In [110]:
list(rain_df.columns)

['SUBDIVISION',
 'YEAR',
 'JAN',
 'FEB',
 'MAR',
 'APR',
 'MAY',
 'JUN',
 'JUL',
 'AUG',
 'SEP',
 'OCT',
 'NOV',
 'DEC',
 'ANNUAL',
 'JF',
 'MAM',
 'JJAS',
 'OND']

In [111]:
crop_df.columns = crop_df.columns.str.strip()
rain_df.columns = rain_df.columns.str.strip()

In [112]:
crop_df['Year'].head()

0    2001-02
1    2002-03
2    2003-04
3    2001-02
4    2002-03
Name: Year, dtype: object

In [113]:
crop_df['Year'] = crop_df['Year'].astype(str).str.extract(r'(\d{4})')
crop_df['Year'] = pd.to_numeric(crop_df['Year'], errors='coerce')

In [114]:
missing_years = crop_df['Year'].isna().sum()
print(missing_years)

0


In [115]:
crop_df = crop_df.dropna(subset=['Year'])
crop_df['Year'] = crop_df['Year'].astype(int)

In [116]:
crop_df.isna().sum()

State                  0
District               0
Crop                  32
Year                   0
Season                 1
Area                  33
Area Units             0
Production          4993
Production Units       0
Yield                 33
dtype: int64

In [117]:
crop_df = crop_df.dropna(subset=['State', 'Crop', 'Production'])

In [118]:
invalid_prod = (crop_df['Production'] <= 0).sum()
print(invalid_prod)
crop_df = crop_df[crop_df['Production'] > 0]

1024


In [119]:
crop_df = crop_df[['State', 'District', 'Crop', 'Year', 'Season', 'Area', 'Production', 'Yield']]

In [120]:
rain_df.columns

Index(['SUBDIVISION', 'YEAR', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL',
       'AUG', 'SEP', 'OCT', 'NOV', 'DEC', 'ANNUAL', 'JF', 'MAM', 'JJAS',
       'OND'],
      dtype='object')

In [121]:
rain_df.rename(columns={'SUBDIVISION': 'State', 'YEAR': 'Year', 'ANNUAL': 'Annual_Rainfall'}, inplace=True)

In [122]:
rain_df = rain_df[['State', 'Year', 'Annual_Rainfall']]

In [123]:
invalid_rain = (rain_df['Annual_Rainfall'] <= 0).sum()
print(invalid_rain)
rain_df = rain_df[rain_df['Annual_Rainfall'] > 0]

0


In [124]:
crop_df['State'].unique()[:5]

array(['Andaman and Nicobar Islands', 'Andhra Pradesh',
       'Arunachal Pradesh', 'Assam', 'Bihar'], dtype=object)

In [125]:
rain_df['State'].unique()[:5]

array(['Andaman & Nicobar Islands', 'Arunachal Pradesh',
       'Assam & Meghalaya', 'Naga Mani Mizo Tripura',
       'Sub Himalayan West Bengal & Sikkim'], dtype=object)

In [126]:
state_mapping = {
    'Andaman & Nicobar Islands': 'Andaman and Nicobar Islands',
}
rain_df['State'] = rain_df['State'].replace(state_mapping)

In [127]:
merged_df = pd.merge(crop_df, rain_df, on=['State', 'Year'], how='inner')
print("Merged dataset:", merged_df.shape)

Merged dataset: (80036, 9)


In [128]:
merged_df.head()

,State,District,Crop,Year,Season,Area,Production,Yield,Annual_Rainfall
0,Andaman and Nicobar Islands,NICOBARS,Arecanut,2001,Kharif,1254.0,2061.0,1.643541,3080.9
1,Andaman and Nicobar Islands,NICOBARS,Arecanut,2002,Whole Year,1258.0,2083.0,1.655803,2620.2
2,Andaman and Nicobar Islands,NICOBARS,Arecanut,2003,Whole Year,1261.0,1525.0,1.209358,2355.9
3,Andaman and Nicobar Islands,NORTH AND MIDDLE ANDAMAN,Arecanut,2001,Kharif,3100.0,5239.0,1.690000,3080.9
4,Andaman and Nicobar Islands,SOUTH ANDAMANS,Arecanut,2002,Whole Year,3105.0,5267.0,1.696296,2620.2


In [129]:
merged_df.to_csv("Merged_Agri_Rainfall_Data.csv", index=False)